# RFM Analysis

This iPython notebook explains how to perform RFM analysis from customer purchase history data. The sample orders file is Sample - Superstore dataset from Tableau Software.

If you have suggestions or improvements please contribute on https://github.com/joaolcorreia/RFM-analysis

In [1]:
import pandas as pd
import warnings
import classification
warnings.filterwarnings('ignore')

Read the sample orders file, containing all past purchases for all customers.

In [2]:
orders = pd.read_csv('Purchase.csv',sep=',')
orders = orders.drop('Third Party Account No', axis=1)
orders['Amount'] = orders['Amount'].abs()
orders['Account No'] = orders['Account No'].astype(int).astype(str)
orders = classification.classify_third_party(orders)

In [3]:
orders.head()

,Date,Timestamp,Account No,Balance,Amount,Third Party Name
0,01/01/2023,00:00,355733816,526.00,664.0,Bank
1,01/01/2023,00:00,472213568,2412.00,600.0,Bank
2,01/01/2023,00:00,624500124,2572.00,654.0,Bank
3,01/01/2023,00:00,203466392,3935.66,672.0,Bank
4,01/01/2023,00:00,768271776,2208.00,632.0,Bank


## Create the RFM Table

Since recency is calculated for a point in time and the Tableau Super Store dataset last order date is Dec 31 2014, that is the date we will use to calculate recency.

Set this date to the current day and extract all orders until yesterday.

In [4]:
import datetime as dt
NOW = dt.datetime(2023,12,31)

In [5]:
# Make the date_placed column datetime
orders['order_date'] = pd.to_datetime(orders['Date'], dayfirst=True)

Create the RFM Table

In [6]:
rfmTable = orders.groupby('Third Party Name').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency
                                        'Account No': lambda x: len(x),      # Frequency
                                        'Amount': lambda x: x.sum()}) # Monetary Value

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'Account No': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)

## Validating the RFM Table

In [7]:
rfmTable.head()

,recency,frequency,monetary_value
Third Party Name,,,
Arts & Craft,25,13842,465253.90
Bank,30,5987,4903077.00
Daily,25,11904,495076.80
Fashion & Clothing,25,55048,4601866.48
Food & Drink,25,22693,450435.19


In [8]:
cu1 = orders[orders['Account No']=='355733816']
cu1

,Date,Timestamp,Account No,Balance,Amount,Third Party Name,order_date
0,01/01/2023,00:00,355733816,526.000000,664.00,Bank,2023-01-01
4361,01/01/2023,10:33,355733816,468.982900,57.02,Grocery,2023-01-01
6552,04/01/2023,09:26,355733816,1271.992900,6.99,Large Retailers,2023-01-04
7107,05/01/2023,11:44,355733816,1243.032900,28.96,Large Retailers,2023-01-05
8458,08/01/2023,09:35,355733816,1236.042900,6.99,Large Retailers,2023-01-08
...,...,...,...,...,...,...,...
206791,30/11/2023,09:27,355733816,6388.106316,21.97,Large Retailers,2023-11-30
207566,01/12/2023,00:00,355733816,5724.106316,664.00,Bank,2023-12-01
212411,02/12/2023,07:33,355733816,5718.073720,6.03,Medical,2023-12-02
213265,03/12/2023,10:52,355733816,5706.093720,11.98,Large Retailers,2023-12-03


## Determining RFM Quartiles

In [9]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [10]:
quantiles

,recency,frequency,monetary_value
0.25,25.0,7466.25,4.541399e+05
0.50,25.0,18267.50,5.404853e+05
0.75,25.0,32976.25,2.348460e+06


Send quantiles to a dictionary, easier to use.

In [11]:
quantiles = quantiles.to_dict()

Recency：
25%的客户最后一次购买是在25天或更少时间之前。
50%的客户（中位数，0.5位置）最后一次购买是在27.5天或更少时间之前。
75%的客户最后一次购买是在126.25天或更少时间之前。

Frequency：
25%的客户在给定时间段内的购买次数是2.25次或更少。
50%的客户的购买次数是1785.5次或更少。
75%的客户的购买次数是3908.25次或更少。

Monetary Value：
25%的客户在给定时间段内的购买金额是324.995或更少。
50%的客户的购买金额是30926.834999999995或更少。
75%的客户的购买金额是135194.74或更少。

In [12]:
quantiles

{'recency': {0.25: 25.0, 0.5: 25.0, 0.75: 25.0},
 'frequency': {0.25: 7466.25, 0.5: 18267.5, 0.75: 32976.25},
 'monetary_value': {0.25: 454139.86749999993,
  0.5: 540485.3350000001,
  0.75: 2348459.635}}

## Creating the RFM segmentation table

In [13]:
rfmSegmentation = rfmTable

We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good. 

In [14]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1


In [15]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [16]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

R_Quartile：基于recency分数的四分位排名，1表示最好（最近的购买），而4表示最差（最久远的购买）。

F_Quartile：基于frequency分数的四分位排名，1表示最差（购买次数最少），而4表示最好（购买次数最多）。

M_Quartile：基于monetary_value分数的四分位排名，1表示最差（消费金额最少），而4表示最好（消费金额最多）。



444：这个客户最近的购买日期较远（R=4），购买频率低（F=4），消费金额也较少（M=4）。这可能表明这个客户对你的业务不再那么活跃。

133：这个客户最近有购买（R=1），但购买频率较低（F=3），消费金额适中（M=3）。

122：这个客户最近有购买（R=1），购买频率高（F=2），消费金额也高（M=2）。这表明这是一个价值较高的客户。

112：这个客户最近有购买（R=1），购买频率非常高（F=1），消费金额也非常高（M=2），这是一个非常重要的客户，可能是一个忠诚并经常大额消费的VIP客户。

In [17]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
Third Party Name,,,,,,,
Arts & Craft,25,13842,465253.90,1,3,3,133
Bank,30,5987,4903077.00,4,4,1,441
Daily,25,11904,495076.80,1,3,3,133
Fashion & Clothing,25,55048,4601866.48,1,1,1,111
Food & Drink,25,22693,450435.19,1,2,4,124


In [18]:
# Uncomment any of the following lines to: copy data to clipboard or save it to a CSV file.
rfmSegmentation.to_clipboard()
rfmSegmentation.to_csv('rfm-table-labelled.csv', sep=',')

Who are the top 5 best customers? by RFM Class (111), high spenders who buy recently and frequently?

In [19]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort_values('monetary_value', ascending=False).head(5)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
Third Party Name,,,,,,,
Fashion & Clothing,25,55048,4601866.48,1,1,1,111
Grocery,25,42604,2896582.54,1,1,1,111
